In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('STUDENT.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 35 columns):
id             1044 non-null int64
InitialName    1044 non-null object
school         992 non-null object
sex            1044 non-null object
age            971 non-null float64
address        1044 non-null object
famsize        1044 non-null object
Pstatus        1044 non-null object
Medu           1044 non-null int64
Fedu           1044 non-null int64
Mjob           1044 non-null object
Fjob           1044 non-null object
reason         428 non-null object
guardian       1044 non-null object
traveltime     1044 non-null int64
studytime      1044 non-null int64
failures       1044 non-null int64
schoolsup      1044 non-null object
famsup         1044 non-null object
paid           1044 non-null object
activities     1044 non-null object
nursery        1044 non-null object
higher         1044 non-null object
internet       1044 non-null object
romantic       1044 non-null object
f

In [4]:
# Task 1.1
df['G3'].value_counts()

PASS    661
FAIL    383
Name: G3, dtype: int64

In [66]:
# Task 1.2
def preprocess():
    import pandas as pd
    #Preprocess data
    df_raw = pd.read_csv('STUDENT.csv', index_col=0)
    df = df_raw.drop(['InitialName', 'guardian'], axis=1)
    
    #Map binaries
    df['address'] = df['address'].map({ 'U':0, 'R':1 })
    df['sex'] = df['sex'].map({ 'M':0, 'F':1 })
    df['famsize'] = df['famsize'].map({ 'LE3':0, 'GT3':1 })
    df['Pstatus'] = df['Pstatus'].map({ 'A':0, 'T':1 })
    df['schoolsup'] = df['schoolsup'].map({ 'no':0, 'yes':1 })
    df['famsup'] = df['famsup'].map({ 'no':0, 'yes':1 })
    df['paid'] = df['paid'].map({ 'no':0, 'yes':1 })
    df['activities'] = df['activities'].map({ 'no':0, 'yes':1 })
    df['nursery'] = df['nursery'].map({ 'no':0, 'yes':1 })
    df['higher'] = df['higher'].map({ 'no':0, 'yes':1 })
    df['internet'] = df['internet'].map({ 'no':0, 'yes':1 })
    df['romantic'] = df['romantic'].map({ 'no':0, 'yes':1 })
    df['G3'] = df['G3'].map({ 'FAIL': 0, 'PASS': 1 })

    #Fill via median for age
    df['age'].fillna(df['age'].median(), inplace=True)
    
    #Fill via 'none' since hot encode will flag
    df['school'].fillna('none', inplace=True)
    df['reason'].fillna('none', inplace=True)

    #Drop g1 & 2 empty rows
    df = df.drop(['G1', 'G2', 'failures'], axis=1)
    #cols_miss_drop =['G1', 'G2']
    #mask = pd.isnull(df['G1'])
    #for col in cols_miss_drop:
    # mask = mask | pd.isnull(df[col])
    #df = df[~mask]

    #Hot encode
    df = pd.get_dummies(df)
    return df

In [67]:
def analyse_feature_importance(dm_model, feature_names, n_to_display=20):
    # grab feature importances from the model
    importances = dm_model.feature_importances_

    # sort them out in descending order
    indices = np.argsort(importances)
    indices = np.flip(indices, axis=0)
    # limit to 20 features, you can leave this out to print out everything
    indices = indices[:n_to_display]
    for i in indices:
        print(feature_names[i], ':', importances[i])
        
def visualize_decision_tree(dm_model, feature_names, save_name):
    dotfile = StringIO()
    export_graphviz(dm_model, out_file=dotfile, feature_names=feature_names)
    graph = pydot.graph_from_dot_data(dotfile.getvalue())
    graph.write_png(save_name) # saved in the following file

In [68]:
df = preprocess()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044 entries, 0 to 1043
Data columns (total 43 columns):
sex                  1044 non-null int64
age                  1044 non-null float64
address              1044 non-null int64
famsize              1044 non-null int64
Pstatus              1044 non-null int64
Medu                 1044 non-null int64
Fedu                 1044 non-null int64
traveltime           1044 non-null int64
studytime            1044 non-null int64
schoolsup            1044 non-null int64
famsup               1044 non-null int64
paid                 1044 non-null int64
activities           1044 non-null int64
nursery              1044 non-null int64
higher               1044 non-null int64
internet             1044 non-null int64
romantic             1044 non-null int64
famrel               1044 non-null int64
freetime             1044 non-null int64
goout                1044 non-null int64
Dalc                 1044 non-null int64
Walc                 1044 non-

In [69]:
df['higher'].value_counts()

1    955
0     89
Name: higher, dtype: int64

In [70]:
df['school_none'].value_counts()

0    992
1     52
Name: school_none, dtype: int64

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

y = df['G3']
x = df.drop(['G3'], axis=1)

#set seed for randomisation
rs = 10

#Convert x into numpy matrix for sklearn consumption
x_mat = x.as_matrix()
#Setup training and test datasets on a 70/30 split
x_train, x_test, y_train, y_test = train_test_split(x_mat, y, test_size=0.3, stratify=y, random_state=rs)

#simple decision tree training
model = DecisionTreeClassifier(random_state=rs)
model.fit(x_train, y_train)

#Check accuracy on the training sets
print("Train accuracy:", model.score(x_train, y_train))
#Check accuracy on the test sets
print("Test accuracy:", model.score(x_test, y_test))

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

#Check which features have the largest impact on the decision tree?
import numpy as np

#grab feature importances from the model and feature name from the original x
importances = model.feature_importances_
feature_names = x.columns

#sort in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

#limit to 20 features
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', importances[i])

Train accuracy: 0.9972602739726028
Test accuracy: 0.589171974522293
             precision    recall  f1-score   support

          0       0.45      0.50      0.47       115
          1       0.69      0.64      0.66       199

avg / total       0.60      0.59      0.59       314

absences : 0.09560916279923631
higher : 0.06658724391357355
age : 0.06262717342173142
Fedu : 0.05760027257597375
traveltime : 0.043278515973278565
freetime : 0.042289114645506304
famrel : 0.041627954427609797
goout : 0.040205962439106256
health : 0.038198143545094314
reason_home : 0.03687757908823
famsup : 0.036806554546183756
Medu : 0.035631807393326664
studytime : 0.0338949196526395
sex : 0.0272595931558402
school_DCHS : 0.026589526320946126
Dalc : 0.025552026252349605
Walc : 0.0235540199840252
internet : 0.022752062565688812
paid : 0.021111823615985976
Mjob_at_home : 0.019885283725948007


C:\Users\longs\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [72]:
import pydot
from io import StringIO
from sklearn.tree import export_graphviz
# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=x.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph.write_png("brendan-default_decision.png") # saved in the following file - will return True if successful

True

In [73]:
from sklearn.model_selection import GridSearchCV

In [74]:
# grid search CV
params = {'criterion': ['gini', 'entropy'],
 'max_depth': range(2, 10),
 'min_samples_leaf': range(20, 60, 10)}
cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(random_state=rs),cv=10)
cv.fit(x_train, y_train)
print("Train accuracy:", cv.score(x_train, y_train))
print("Test accuracy:", cv.score(x_test, y_test))
# test the best model
y_pred = cv.predict(x_test)
print(classification_report(y_test, y_pred))
# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.7219178082191781
Test accuracy: 0.6592356687898089
             precision    recall  f1-score   support

          0       0.54      0.50      0.52       115
          1       0.72      0.75      0.74       199

avg / total       0.65      0.66      0.66       314

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 30}


In [75]:
# grid search CV
params = {'criterion': ['gini', 'entropy'],
 'max_depth': range(1, 15),
 'min_samples_leaf': range(24, 37)}
cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(random_state=rs),cv=10)
cv.fit(x_train, y_train)
print("Train accuracy:", cv.score(x_train, y_train))
print("Test accuracy:", cv.score(x_test, y_test))
# test the best model
y_pred = cv.predict(x_test)
print(classification_report(y_test, y_pred))
# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.7178082191780822
Test accuracy: 0.643312101910828
             precision    recall  f1-score   support

          0       0.51      0.51      0.51       115
          1       0.72      0.72      0.72       199

avg / total       0.64      0.64      0.64       314

{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 34}


In [65]:
analyse_feature_importance(cv.best_estimator_, x.columns, 20)

failures : 1.0
schoolsup : 0.0
famrel : 0.0
romantic : 0.0
internet : 0.0
higher : 0.0
nursery : 0.0
activities : 0.0
paid : 0.0
famsup : 0.0
reason_reputation : 0.0
goout : 0.0
studytime : 0.0
traveltime : 0.0
Fedu : 0.0
Medu : 0.0
Pstatus : 0.0
famsize : 0.0
address : 0.0
age : 0.0


In [39]:
visualize_decision_tree(cv.best_estimator_, x.columns, "brendan-optimal_decision.png")

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [41]:
x_train_scaler = scaler.fit_transform(x_train, y_train)

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
model = LogisticRegression(random_state=rs)
# fit it to training data
model.fit(x_train_scaler, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [44]:
# training and test accuracy
print("Train accuracy:", model.score(x_train_scaler, y_train))
print("Test accuracy:", model.score(x_test, y_test))
# classification report on test data
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

Train accuracy: 0.7342465753424657
Test accuracy: 0.3662420382165605
             precision    recall  f1-score   support

          0       0.37      1.00      0.54       115
          1       0.00      0.00      0.00       199

avg / total       0.13      0.37      0.20       314



C:\Users\longs\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
feature_names = x.columns
coef = model.coef_[0]
# limit to 20 features, you can comment the following line to print out everything
coef = coef[:20]
for i in range(len(coef)):
    print(feature_names[i], ':', coef[i])

sex : 0.2431274764602015
age : -0.26578375258821146
address : -0.004342089777775582
famsize : -0.061954211352185325
Pstatus : -0.05565581730388714
Medu : 0.1737062742216046
Fedu : 0.3335076804114587
traveltime : -0.0935315069961461
studytime : 0.2802068924590801
schoolsup : -0.3552837191624801
famsup : -0.21740670296107673
paid : -0.20041959509309853
activities : 0.011872225521872065
nursery : -0.0750086382071716
higher : 0.36049854887312976
internet : 0.042183867408176486
romantic : -0.14827023611725745
famrel : 0.14398854136151648
freetime : -0.08946854746848248
goout : -0.0476429877690482


In [46]:
# grab feature importances from the model and feature name from the original X
coef = model.coef_[0]
feature_names = x.columns
# sort them out in descending order
indices = np.argsort(np.absolute(coef))
indices = np.flip(indices, axis=0)
# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]
for i in indices:
    print(feature_names[i], ':', coef[i])

higher : 0.36049854887312976
schoolsup : -0.3552837191624801
Fedu : 0.3335076804114587
studytime : 0.2802068924590801
age : -0.26578375258821146
absences : -0.26574362828873427
sex : 0.2431274764602015
famsup : -0.21740670296107673
paid : -0.20041959509309853
school_THS : -0.18128806217421284
Medu : 0.1737062742216046
school_DCHS : 0.17076209462230546
Walc : -0.1641437185405883
Fjob_health : -0.1526297769099963
romantic : -0.14827023611725745
famrel : 0.14398854136151648
reason_other : 0.13591318978510697
reason_course : -0.12386815576199112
Fjob_services : -0.11221469068465104
reason_home : -0.09790106141375353


In [47]:
# grid search CV
params = {'C': [pow(10, x) for x in range(-8, 6)]}
# use all cores to tune logistic regression with C parameter
cv = GridSearchCV(param_grid=params, estimator=LogisticRegression(random_state=rs),
cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
# test the best model
print("Train accuracy:", cv.score(x_train_scaler, y_train))
print("Test accuracy:", cv.score(x_test, y_test))
y_pred = cv.predict(x_test)
print(classification_report(y_test, y_pred))
# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.7
Test accuracy: 0.6942675159235668
             precision    recall  f1-score   support

          0       0.62      0.43      0.51       115
          1       0.72      0.84      0.78       199

avg / total       0.68      0.69      0.68       314

{'C': 0.1}
